In [26]:
import pandas as pd

In [27]:
wrds = pd.read_csv('../../data/raw/wrds.csv')

In [28]:
wrds['ITEM6008']

0        ARP096691133
1        ARP096691133
2        ARP096691133
3        ARP096691133
4        ARP096691133
             ...     
30244    JP3389660006
30245    JP3389660006
30246    JP3389660006
30247    JP3389660006
30248    JP3389660006
Name: ITEM6008, Length: 30249, dtype: object

In [32]:
wrds.columns

Index(['code', 'year_', 'freq', 'ITEM6008', 'ITEM7011', 'ITEM7210', 'ITEM7220',
       'ITEM7230', 'ITEM7240', 'ITEM7250', 'ITEM8301', 'ITEM8601', 'ITEM8621',
       'ITEM8625', 'ITEM8626', 'ITEM8630', 'ITEM8636', 'ITEM6003'],
      dtype='object')

In [9]:
colnames = ['code', 'year', 'freq', 'isin', 'employees', 'mkt cap', 'common equity',
       'tot assets', 'revenue', 'net income', 'roe', 'eps 1yr gr', 'assets 1y gr',
       'assets 5 yr gr', 'employees 1y gr', 'employees 5y gr', 'net income 1y gr', 'name']

In [10]:
# Rename columns
wrds.columns = colnames

In [11]:
wrds.drop('freq', axis=1, inplace=True)

In [12]:
# drop 2023 
wrds = wrds[wrds.year != 2023]

In [13]:
wrds['cdp_year'] = wrds['year'] + 1

In [14]:
# group by isin select first and count percentage of nans by column
wrds.groupby('isin').first().isna().sum() / wrds.groupby('isin').first().shape[0]

code                0.000000
year                0.000000
employees           0.112454
mkt cap             0.014797
common equity       0.031073
tot assets          0.025154
revenue             0.025154
net income          0.025154
roe                 0.032799
eps 1yr gr          0.052774
assets 1y gr        0.025401
assets 5 yr gr      0.035758
employees 1y gr     0.117633
employees 5y gr     0.151171
net income 1y gr    0.068311
name                0.000000
cdp_year            0.000000
dtype: float64

In [15]:
# drop employees 5y gr
wrds.drop(['employees 5y gr'], axis=1, inplace=True)

In [16]:
# Create a column for the count of NaNs in each row
wrds['NaN_Count'] = wrds.apply(lambda x: x.isna().sum(), axis=1)

# Sort by 'Year', 'ISIN', and 'NaN_Count' (ascending order)
wrds.sort_values(by=['year', 'isin', 'NaN_Count'], inplace=True)

# Drop duplicates based on 'Year' and 'ISIN', keeping the first (which has fewer NaNs)
wrds.drop_duplicates(subset=['year', 'isin'], keep='first', inplace=True)

In [17]:
# Group the DataFrame by ISIN
grouped = wrds.groupby('isin')

# Calculate the percentage of groups with at least one NaN in each column
percentage_nan = {}
for column in wrds.columns:
    if column != 'isin':
        percentage_nan[column] = grouped[column].apply(lambda x: x.isna().any()).mean() * 100

percentage_nan

{'code': 0.0,
 'year': 0.0,
 'employees': 23.42786683107275,
 'mkt cap': 10.110974106041924,
 'common equity': 9.173859432799015,
 'tot assets': 7.200986436498151,
 'revenue': 7.028360049321824,
 'net income': 7.0530209617755855,
 'roe': 12.799013563501848,
 'eps 1yr gr': 44.611590628853264,
 'assets 1y gr': 10.776818742293464,
 'assets 5 yr gr': 15.141800246609124,
 'employees 1y gr': 25.696670776818742,
 'net income 1y gr': 45.252774352651045,
 'name': 0.0,
 'cdp_year': 0.0,
 'NaN_Count': 0.0}

Problem with duplicated rows found:

In [18]:
percentage_nan = {}
for column in wrds.columns:
    if column != 'isin':
        percentage_nan[column] = grouped[column].apply(lambda x: x.isna().sum() > 2).mean() * 100

percentage_nan

{'code': 0.0,
 'year': 0.0,
 'employees': 13.045622688039456,
 'mkt cap': 2.244143033292232,
 'common equity': 2.7127003699136867,
 'tot assets': 1.8249075215782986,
 'revenue': 1.8249075215782986,
 'net income': 1.8249075215782986,
 'roe': 3.847102342786683,
 'eps 1yr gr': 20.07398273736128,
 'assets 1y gr': 2.1454993834771887,
 'assets 5 yr gr': 6.485819975339087,
 'employees 1y gr': 14.574599260172628,
 'net income 1y gr': 21.479654747225645,
 'name': 0.0,
 'cdp_year': 0.0,
 'NaN_Count': 0.0}

In [19]:
wrds

,code,year,isin,employees,mkt cap,common equity,tot assets,revenue,net income,roe,eps 1yr gr,assets 1y gr,assets 5 yr gr,employees 1y gr,net income 1y gr,name,cdp_year,NaN_Count
23068,71093,2017,AEA002001013,NaN,4.709495e+09,6.267848e+09,9.909427e+09,1.682751e+09,5.432962e+08,8.88778,-28.27875,2.35040,2.58232,NaN,-28.28023,ALDAR PROPERTIES,2018,2
22753,69606,2017,AEN000101016,NaN,3.028070e+10,2.473738e+10,1.821233e+11,6.066939e+09,2.382699e+09,13.41216,-12.35756,58.92320,NaN,NaN,48.63457,FIRST ABU DHABI BANK,2018,3
17049,40721,2017,AN8068571086,100000.0,9.326776e+10,3.684200e+10,7.198700e+10,3.044000e+10,-1.505000e+09,-3.86294,NaN,-7.65688,3.18331,0.00000,NaN,SCHLUMBERGER NV,2018,2
2,577,2017,ARP096691133,11231.0,NaN,NaN,1.807052e+10,3.518464e+09,3.714654e+08,25.27051,33.02698,40.12620,39.89902,-2.66077,35.75288,BANCO DE GALICIA,2018,2
23209,71837,2017,ARP432631215,NaN,5.569515e+09,9.084052e+08,5.541809e+09,2.704641e+09,2.474343e+08,32.94235,NaN,35.65783,55.78444,NaN,NaN,PAMPA ENERGIA,2018,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11237,21432,2022,ZAE000248811,4609.0,2.316811e+09,9.626963e+08,1.801715e+09,2.096824e+09,1.460267e+08,16.53168,22.65265,14.17076,7.62403,3.80631,22.79548,DISTELL GROUP,2023,0
14052,26563,2022,ZAE000255360,31866.0,3.021427e+09,3.752493e+09,6.101437e+10,8.154752e+09,4.384372e+08,12.13797,11.02346,5.56558,-22.58858,10.50387,11.27831,OLD MUTUAL LTD,2023,0
11056,20929,2022,ZAE000255915,35451.0,9.430542e+09,7.905675e+09,1.049027e+11,9.456703e+09,1.208223e+09,15.55082,15.82970,9.24509,8.91043,0.52173,15.71807,ABSA GROUP LTD,2023,0
29230,137665,2022,ZAE000261913,17283.0,1.127256e+09,8.553513e+08,2.563904e+09,5.650209e+09,2.021047e+08,25.33107,64.97410,12.28377,4.33041,3.44147,56.81602,MOTUS HOLDINGS LTD,2023,0


In [20]:
wrds = wrds.dropna()

In [23]:
wrds['isin'].nunique()

3325

In [378]:
summary = pd.read_csv("../../data/processed/cdp_summary_selected.csv")

In [379]:
# rename year to cdp_year
summary.rename(columns={'year': 'cdp_year'}, inplace=True)

In [380]:
summary

,id,cdp_year,isin,reporting_boundary,country,industry14,industry46,cdpdate
0,44,2018.0,GB00B1YW4409,operational control,United Kingdom,Services,banks diverse financials insurance,8.0
1,44,2019.0,GB00B1YW4409,operational control,United Kingdom,Services,banks diverse financials insurance,7.0
2,44,2020.0,GB00B1YW4409,operational control,United Kingdom,Services,banks diverse financials insurance,8.0
3,44,2021.0,GB00B1YW4409,operational control,United Kingdom,Services,banks diverse financials insurance,7.0
4,44,2022.0,GB00B1YW4409,operational control,United Kingdom,Services,banks diverse financials insurance,7.0
...,...,...,...,...,...,...,...,...
11784,895420,2022.0,KYG2R55E1030,operational control,Hong Kong,Power generation,not reported,8.0
11785,895480,2022.0,US6031581068,operational control,USA,Materials,not reported,7.0
11786,895556,2022.0,US53960E1064,not reported,USA,"Food, beverage & agriculture",not reported,7.0
11787,895582,2022.0,MYL5211OO007,operational control,Malaysia,Infrastructure,not reported,8.0


In [381]:
# merge the two dataframes on isin and year = cdp_year
test = pd.merge(wrds, summary, how='inner', on=['isin', 'cdp_year'])

In [382]:
test.columns

Index(['code', 'year', 'isin', 'employees', 'mkt cap', 'common equity',
       'tot assets', 'revenue', 'net income', 'roe', 'eps 1yr gr',
       'assets 1y gr', 'assets 5 yr gr', 'employees 1y gr', 'net income 1y gr',
       'name', 'cdp_year', 'NaN_Count', 'id', 'reporting_boundary', 'country',
       'industry14', 'industry46', 'cdpdate'],
      dtype='object')

In [383]:
test

,code,year,isin,employees,mkt cap,common equity,tot assets,revenue,net income,roe,...,net income 1y gr,name,cdp_year,NaN_Count,id,reporting_boundary,country,industry14,industry46,cdpdate
0,59457,2017,AT0000606306,49700.0,1.189693e+10,1.191486e+10,1.619088e+11,8.510987e+09,1.314786e+09,11.79842,...,136.77878,RAIFFEISEN BANK,2018,0,15493,operational control,Austria,Services,banks diverse financials insurance,7.0
1,1754,2017,AT0000720008,18957.0,6.154327e+09,2.809868e+09,8.766421e+09,5.131565e+09,3.826748e+08,14.11867,...,-17.64228,TELEKOM AUSTRIA AG,2018,0,18433,financial control,Austria,Services,telecommunication services,8.0
2,1769,2017,AT0000741053,6840.0,2.776054e+09,3.414529e+09,7.526934e+09,2.615826e+09,2.963406e+08,9.29113,...,60.48593,EVN AG,2018,0,6086,equity share,Austria,Infrastructure,electric utilities independent power producers...,8.0
3,1788,2017,AT0000746409,2819.0,8.391708e+09,6.072000e+09,1.352949e+10,3.493100e+09,3.614386e+08,6.06175,...,-28.97652,VERBUND AG,2018,0,20108,operational control,Austria,Infrastructure,electric utilities independent power producers...,8.0
4,1820,2017,AT0000758305,10212.0,1.534606e+09,6.509804e+08,1.834709e+09,1.763878e+09,6.296519e+07,9.57056,...,-14.15657,PALFINGER AG,2018,0,31965,other,Austria,Manufacturing,electrical equipment machinery,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7633,20910,2021,ZAE000134854,15871.0,5.117490e+09,3.309817e+08,1.175593e+09,2.571912e+09,1.265915e+08,36.76027,...,-2.25080,CLICKS GROUP LIMITED,2022,0,13060,operational control,South Africa,Retail,food staples retailing,8.0
7634,20873,2021,ZAE000191342,7711.0,2.822350e+08,3.155197e+08,6.777978e+08,4.900358e+08,8.049594e+08,279.14561,...,1775.61728,ALTRON LTD,2022,0,597,operational control,South Africa,Retail,not reported,8.0
7635,21432,2021,ZAE000248811,4440.0,2.573941e+09,9.167429e+08,1.799529e+09,1.979231e+09,1.356056e+08,15.69394,...,519.86551,DISTELL GROUP,2022,0,4816,equity share,South Africa,"Food, beverage & agriculture",not reported,7.0
7636,137665,2021,ZAE000261913,16708.0,1.195599e+09,8.442426e+08,2.603829e+09,6.108710e+09,1.469649e+08,17.08469,...,585.62092,MOTUS HOLDINGS LTD,2022,0,833352,operational control,South Africa,Retail,not reported,8.0


In [384]:
# read testing
response = pd.read_csv('../../data/processed/boolean_response.csv')

In [385]:
# rename year to cdp_year
response.rename(columns={'year': 'cdp_year'}, inplace=True)

In [386]:
# shift year down by one
response['cdp_year'] = response['cdp_year'] - 1

In [387]:
# merge the two dataframes on isin and year = cdp_year
sec = pd.merge(test, response, how='inner', on=['id', 'cdp_year'])

In [388]:
sec.columns

Index(['code', 'year', 'isin', 'employees', 'mkt cap', 'common equity',
       'tot assets', 'revenue', 'net income', 'roe', 'eps 1yr gr',
       'assets 1y gr', 'assets 5 yr gr', 'employees 1y gr', 'net income 1y gr',
       'name', 'cdp_year', 'NaN_Count', 'id', 'reporting_boundary', 'country',
       'industry14', 'industry46', 'cdpdate', 'ghg_change'],
      dtype='object')

In [389]:
sec.drop(['code', 'isin', 'name', 'NaN_Count'], axis=1, inplace=True)

In [390]:
sec

,year,employees,mkt cap,common equity,tot assets,revenue,net income,roe,eps 1yr gr,assets 1y gr,...,employees 1y gr,net income 1y gr,cdp_year,id,reporting_boundary,country,industry14,industry46,cdpdate,ghg_change
0,2017,49700.0,1.189693e+10,1.191486e+10,1.619088e+11,8.510987e+09,1.314786e+09,11.79842,110.78652,20.86412,...,2.35604,136.77878,2018,15493,operational control,Austria,Services,banks diverse financials insurance,7.0,decrease
1,2017,18957.0,6.154327e+09,2.809868e+09,8.766421e+09,5.131565e+09,3.826748e+08,14.11867,-17.64232,-4.51317,...,4.14217,-17.64228,2018,18433,financial control,Austria,Services,telecommunication services,8.0,increase
2,2017,6840.0,2.776054e+09,3.414529e+09,7.526934e+09,2.615826e+09,2.963406e+08,9.29113,60.40213,-1.25000,...,0.14641,60.48593,2018,6086,equity share,Austria,Infrastructure,electric utilities independent power producers...,8.0,decrease
3,2017,2819.0,8.391708e+09,6.072000e+09,1.352949e+10,3.493100e+09,3.614386e+08,6.06175,-28.97697,-2.20646,...,-4.50542,-28.97652,2018,20108,operational control,Austria,Infrastructure,electric utilities independent power producers...,8.0,decrease
4,2017,10212.0,1.534606e+09,6.509804e+08,1.834709e+09,1.763878e+09,6.296519e+07,9.57056,-14.46094,0.81893,...,2.42728,-14.15657,2018,31965,other,Austria,Manufacturing,electrical equipment machinery,8.0,same
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5288,2020,29776.0,8.780444e+08,8.929450e+08,2.292517e+09,1.978458e+09,1.368772e+08,16.15741,-7.54813,41.31654,...,2.24924,-7.43532,2021,31545,operational control,South Africa,Retail,retailing,7.0,increase
5289,2020,10593.0,1.150159e+09,1.348526e+09,2.169856e+09,9.104682e+08,2.401484e+08,19.60376,5121.97530,18.56912,...,5.01636,5405.46022,2021,31343,operational control,South Africa,Materials,not reported,7.0,increase
5290,2020,7331.0,4.785590e+08,2.513870e+08,8.609956e+08,1.066624e+09,4.275940e+07,17.92882,-5.82296,17.86650,...,-16.05405,-5.76653,2021,597,operational control,South Africa,Retail,not reported,7.0,increase
5291,2020,4845.0,9.674566e+08,6.685665e+08,1.455069e+09,1.291209e+09,1.802596e+07,2.69259,-65.18582,7.57237,...,-1.84360,-65.17016,2021,4816,equity share,South Africa,"Food, beverage & agriculture",not reported,7.0,decrease


In [391]:
# export sec as a first classification model 
sec.to_csv('../../data/processed/sec.csv', index=False)

In [392]:
sec.columns

Index(['year', 'employees', 'mkt cap', 'common equity', 'tot assets',
       'revenue', 'net income', 'roe', 'eps 1yr gr', 'assets 1y gr',
       'assets 5 yr gr', 'employees 1y gr', 'net income 1y gr', 'cdp_year',
       'id', 'reporting_boundary', 'country', 'industry14', 'industry46',
       'cdpdate', 'ghg_change'],
      dtype='object')